<a href="https://colab.research.google.com/github/PJunior17/Iris-Neural-Network/blob/main/Iris_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Load and Prepare the Dataset

### Load the Dataset

In [30]:
iris = load_iris()
X = iris.data #is a matrix of the input features
y = iris.target #is a list of all the output features as 0,1,2

X, y

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
        [5

### Train Test Split

In [18]:
# train_test_split take the input features, output features, the percentage of test data and a random seed to shuffle the data
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=17)


### Standardize Features

In [21]:
#standardize the features
scalar = StandardScaler() #Standardize features by removing the mean and scaling to unit variance
Xtrain = scalar.fit_transform(Xtrain) #Fit to data, then transform it
Xtest = scalar.transform(Xtest) #Perform standardization by centering and scaling

Xtest

array([[-0.51974235,  1.8916694 , -1.32816828, -0.99726286],
       [ 0.1771189 , -0.36704033,  0.45510661,  0.44456296],
       [ 0.9901237 , -0.14116936,  0.845198  ,  1.49316355],
       [-0.17131172, -0.59291131,  0.23219725,  0.18241281],
       [ 0.40940599, -0.59291131,  0.62228864,  0.83778818],
       [ 0.29326244, -0.14116936,  0.67801598,  0.83778818],
       [-0.51974235, -0.14116936,  0.45510661,  0.44456296],
       [ 2.15155912,  1.66579843,  1.68110811,  1.36208848],
       [-0.40359881, -1.4963952 ,  0.06501523, -0.07973734],
       [-0.17131172, -1.27052423,  0.73374332,  1.09993833],
       [ 0.9901237 ,  0.53644356,  1.1238347 ,  1.7553137 ],
       [-1.10046006,  0.08470162, -1.2167136 , -1.39048808],
       [-0.17131172, -0.59291131,  0.45510661,  0.18241281],
       [-1.2166036 ,  0.76231454, -1.16098626, -1.259413  ],
       [ 0.75783661,  0.31057259,  0.78947066,  1.09993833],
       [-0.86817298,  1.66579843, -1.2167136 , -1.12833793],
       [-1.68117777, -0.

### Convert to Tensors

In [31]:
Xtrain_tensor = torch.tensor(Xtrain, dtype=torch.float32)
Xtest_tensor = torch.tensor(Xtest, dtype=torch.float32)
ytrain_tensor = torch.tensor(ytrain, dtype=torch.long) #need to do long because the target is full of ints and not floats
ytest_tensor = torch.tensor(ytest, dtype=torch.long)

type(Xtest_tensor), Xtest_tensor.dtype, type(ytrain_tensor), ytrain_tensor.dtype

(torch.Tensor, torch.float32, torch.Tensor, torch.int64)

## Create the Model

In [32]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.fc1 = nn.Linear(4,10) # the first number is the number of input features you have and the second is how many neurons is it going to
    self.fc2 = nn.Linear (10, 3) #this is the next layer and it takes all 10 outputs from the last layer and spits out 3 outputs which is our target

  #when using nn.Module we have to also define the forward propagation method forward()
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    return x


In [34]:
model = Model() #instantiate the model
loss_fn = nn.CrossEntropyLoss() #loss function we are using
optimizer = optim.Adam(model.parameters(), lr=1e-2) #gradient descent algorithm

## Training the Model

In [44]:
epochs = 100

for epoch in range(epochs):
  model.train() #telling pytorch that we are changing the mode to change
  optimizer.zero_grad() #we are clearing the gradient descent memory so it doesn't carry through each iteration, this helps with processing and not interfering with data
  outputs = model(Xtrain_tensor) #forward propagation
  loss = loss_fn(outputs, ytrain_tensor) #loss function
  loss.backward() #back propagation
  optimizer.step() #this updates the parameters based on the gradient descent

  print('Epoch: %s | Loss: %s' % (epoch, loss))

Epoch: 0 | Loss: tensor(0.0287, grad_fn=<NllLossBackward0>)
Epoch: 1 | Loss: tensor(0.0286, grad_fn=<NllLossBackward0>)
Epoch: 2 | Loss: tensor(0.0286, grad_fn=<NllLossBackward0>)
Epoch: 3 | Loss: tensor(0.0286, grad_fn=<NllLossBackward0>)
Epoch: 4 | Loss: tensor(0.0286, grad_fn=<NllLossBackward0>)
Epoch: 5 | Loss: tensor(0.0285, grad_fn=<NllLossBackward0>)
Epoch: 6 | Loss: tensor(0.0285, grad_fn=<NllLossBackward0>)
Epoch: 7 | Loss: tensor(0.0285, grad_fn=<NllLossBackward0>)
Epoch: 8 | Loss: tensor(0.0285, grad_fn=<NllLossBackward0>)
Epoch: 9 | Loss: tensor(0.0284, grad_fn=<NllLossBackward0>)
Epoch: 10 | Loss: tensor(0.0284, grad_fn=<NllLossBackward0>)
Epoch: 11 | Loss: tensor(0.0284, grad_fn=<NllLossBackward0>)
Epoch: 12 | Loss: tensor(0.0284, grad_fn=<NllLossBackward0>)
Epoch: 13 | Loss: tensor(0.0284, grad_fn=<NllLossBackward0>)
Epoch: 14 | Loss: tensor(0.0283, grad_fn=<NllLossBackward0>)
Epoch: 15 | Loss: tensor(0.0283, grad_fn=<NllLossBackward0>)
Epoch: 16 | Loss: tensor(0.0283, g

## Evaluate the Model

In [50]:
with torch.no_grad():
    model.eval()
    outputs = model(Xtest_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(ytest_tensor.numpy(), predicted.numpy())
    print('Accuracy on test set: %s' % (accuracy))

Accuracy on test set: 0.9666666666666667
